# ChatGPT API Zero Shot Sentiment Analysis on User-Defined Emotions

by Max Woolf ([@minimaxir](https://twitter.com/minimaxir))

This Colab Notebook easily demonstrates how to build zero shot sentiment analysis using the ChatGPT API! The core system prompt is hackable if you want to extend this to other classification use cases.

This ChatGPT API sentiment analyzer requires an OpenAI account with a payment method attached to it/a free trial, and an [OpenAI API Key](https://platform.openai.com/account/api-keys). Running the setup cells by **mousing over the cells and pressing the Play button** will prompt you to input a key from that link and press Enter; it will not be saved to the Notebook.

Each request is roughly 90 ChatGPT tokens for small input texts, costing $0.18 for 1,000 requests.

## Setup

In [1]:
!pip install -q openai tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 KB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 KB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 23.4 MB/s eta 0:00:00


In [2]:
import openai
import os
import tiktoken
import getpass

api_key = getpass.getpass("Enter the OpenAI API Key: ")
assert api_key.startswith("sk-"), 'OpenAI API Keys begin with "sk-".'
openai.api_key = api_key

Enter the OpenAI API Key: ··········


In [4]:
enc = tiktoken.encoding_for_model("gpt-3.5-turbo")

def create_logit_biases(emotions, bias_weight):
    """Creates a dict of tokens for ChatGPT to use mostly exclusively."""
    bias_dict = {}
    for emotion in emotions:
        token_ids = enc.encode(emotion)
        for token_id in token_ids:
            bias_dict[token_id] = bias_weight
    return bias_dict

def chatgpt_sentiment(prompt, emotions=None):
    if not emotions:
        emotions = globals().get("emotions")
    emotion_list_str = [f"- {x}\n" for x in emotions]
    eos_str = "."
    system = f"You are an emotionally intelligent assistant. Classify the sentiment of the user's text with ONLY ONE OF THE FOLLOWING EMOTIONS:\n{''.join(emotion_list_str)}\n\nAfter classifying a text, respond with \"{eos_str}\"."
    r = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": system},
            {"role": "user", "content": prompt},
        ],
        stop=eos_str,
        max_tokens=20,  # safeguard from going infinite
        temperature=0.0,  # deterministic, will use largest logit
        logit_bias=create_logit_biases(
            emotions + [eos_str], 15  # may want to tweak bias weight
        ),
    )

    result = r["choices"][0]["message"]["content"]
    if result == "":  # if ChatGPT decides to not return an emotion
        result = "N/A"
    return result

## Generate The Sentiment

The `emotions` list indicates the range of emotions ChatGPT is able to choose from for classification. Change the emotions if you want, and run the cell when done.

In the case a text is _extremely_ neutral, an `N/A` will be returned instead of the emotion.

_Protip_: Due to how tokenization works, it's better to use consistent modifiers like `very` and also not use longer/less common emotions, e.g. `very happy` is better than `ecstatic`.

In [5]:
emotions = [
    "happy",
    "sad",
    "angry",
    "tired",
    "very happy",
    "very sad",
    "very angry",
    "very tired",
]

Input the text you want to classify in the cell below, then run the cell!

In [6]:
text = 'Artificial intelligence is fun! :D' #@param {type:"string"}
chatgpt_sentiment(text)

'happy'

## Test Cases

These test cases were run with the initial set of 8 emotions, but feel free to rerun them on your own set of emotions and see what happens! 

In [7]:
chatgpt_sentiment("I just bought a new iPhone.")

'happy'

In [8]:
chatgpt_sentiment("I want to marry you!")

'very happy'

In [9]:
chatgpt_sentiment("Feeling weird today.")

'tired'

In [10]:
chatgpt_sentiment("I have literally no energy.")

'very tired'

In [11]:
chatgpt_sentiment("I can't believe I got fired!")

'sad'

In [12]:
chatgpt_sentiment("My mother just died from cancer.")

'very sad'

In [13]:
chatgpt_sentiment("I am mad")

'angry'

In [14]:
chatgpt_sentiment("GOD DAMN IT")

'very angry'

In [15]:
chatgpt_sentiment("idk lol")

'N/A'

In [16]:
chatgpt_sentiment("😀")

'very happy'

In [17]:
chatgpt_sentiment("😞")

'sad'

In [18]:
chatgpt_sentiment("😡")

'angry'

In [19]:
chatgpt_sentiment("😪")

'tired'

In [20]:
chatgpt_sentiment("🧜‍♂️")

'N/A'

## MIT License

Copyright (c) 2023 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
